In [9]:
from sqlalchemy import create_engine, inspect
import subprocess
import pandas as pd
import geopandas as gpd
import psycopg2
from psycopg2 import sql
import plotly.express as px
import matplotlib.pyplot as plt
import time
import sys
import folium
import os

# move current directory to /Users/tanyatsui/Documents/01_Projects/housingEmissions
os.chdir('/Users/tanyatsui/Documents/01_Projects/housingEmissions')

In [10]:
# create a connection to the database
db_name = 'urbanmining'
db_user = 'postgres'
db_password = 'Tunacompany5694!'
db_host = 'localhost'
db_port = '5432'
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Fixing admin boundary adder
- 2% of buildings in `bag_pand` don't have a `wk_code` 
- 1% of units in `bag_vbo` don't have a `wk_code` 
- There are 52 wk_codes missing from the buurt to wijk key `key_buurt2022_to_wijk2012` 

I've decided: the problem is too small to fix.

### Visualising the issue

In [11]:
base_query = ''' 
WITH cbs_wijk AS (
	SELECT wk_code, geom FROM cbs_wijk_2012 WHERE water = 'NEE'
), 
key AS (
	SELECT DISTINCT wk_code FROM key_buurt2022_to_wijk2012
), 
problem_wijken AS (
	SELECT a.wk_code, ST_Transform(a.geom, 4326) AS geom
	FROM cbs_wijk a 
	LEFT JOIN key b 
	ON a.wk_code = b.wk_code 
	WHERE b.wk_code IS NULL
), 
problem_panden_sample AS (
	SELECT id_pand, ST_Centroid(geom) AS geom FROM bag_pand WHERE wk_code IS NULL LIMIT 100
)
'''

query_problem_wijken = ''' 
SELECT * FROM problem_wijken
'''

query_problem_panden_sample = '''
SELECT * FROM problem_panden_sample
'''

problem_wijken = gpd.read_postgis(base_query + query_problem_wijken, engine, geom_col='geom')
problem_panden_sample = gpd.read_postgis(base_query + query_problem_panden_sample, engine, geom_col='geom')

In [16]:
# plot problem_wijken in folium
m = folium.Map(location=[52.379189, 4.899431], zoom_start=11)
folium.GeoJson(problem_wijken).add_to(m)
for _, row in problem_panden_sample.iterrows():
    folium.CircleMarker(
        location=[row.geom.y, row.geom.x],  # Extract latitude and longitude
        radius=2,  # Radius of the circle
        color='red',  # Circle border color
        fill=True,
        fill_color='red',  # Circle fill color
        fill_opacity=0.6
    ).add_to(m)
m